<a href="https://colab.research.google.com/github/zwcrowley/module_21_deep_learning_challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Module 21- Deep Learning Challenge**

# Part 3: Optimize the Model

#### By: Zack Crowley


## Preprocessing

In [115]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#  Import and read the charity_data.csv on local machine:
# application_df = pd.read_csv("../Resources/charity_data.csv")

In [132]:
# Import from google drive folder:
# Mount google drive to get data:
from google.colab import drive
drive.mount('/content/gdrive')
app_filepath = "/content/gdrive/MyDrive/DataClassNotebooks/module_21_deep_learning_challenge/Resources/charity_data.csv"
# Read in data using pandas:
application_df = pd.read_csv(app_filepath)
application_df.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [133]:
# Check for NAs, duplicates and get the shape of the data:
print(f'The shape of the application data is: {application_df.shape}')
print(f'The number of NAs in the application data: {application_df.isnull().sum()}')
print(f'The duplicate rows of NAs in the application data: {application_df.duplicated().sum()}')
# There are 15 columns, 618 rows, and no NAs or duplicates in the application dataset.
# 9 features and 1 target (IS_SUCCESSFUL).

The shape of the application data is: (34299, 12)
The number of NAs in the application data: EIN                       0
NAME                      0
APPLICATION_TYPE          0
AFFILIATION               0
CLASSIFICATION            0
USE_CASE                  0
ORGANIZATION              0
STATUS                    0
INCOME_AMT                0
SPECIAL_CONSIDERATIONS    0
ASK_AMT                   0
IS_SUCCESSFUL             0
dtype: int64
The duplicate rows of NAs in the application data: 0


In [134]:
# Numeric variable stats
application_df.describe()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL
count,3.429900e+04,34299.000000,3.429900e+04,34299.000000
mean,5.191852e+08,0.999854,2.769199e+06,0.532406
std,2.451472e+08,0.012073,8.713045e+07,0.498956
min,1.052060e+07,0.000000,5.000000e+03,0.000000
25%,2.748482e+08,1.000000,5.000000e+03,0.000000
50%,4.656317e+08,1.000000,5.000000e+03,1.000000
75%,7.526117e+08,1.000000,7.742000e+03,1.000000
max,9.960869e+08,1.000000,8.597806e+09,1.000000


In [135]:
# Check out STATUS column:
print(f'Count of values for STATUS column: \n{application_df["STATUS"].value_counts()}')
#  Looks like STATUS is not going to be very useful to a nn model, probably take it out this time through:

Count of values for STATUS column: 
1    34294
0        5
Name: STATUS, dtype: int64


In [136]:
# Check out SPECIAL_CONSIDERATIONS column:
application_df["SPECIAL_CONSIDERATIONS"].value_counts()
#  Only 27 rows have yest in SPECIAL_CONSIDERATIONS, probably take it out this time through as well.

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [137]:
# Get the column names:
list(application_df.columns)


['EIN',
 'NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'STATUS',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS',
 'ASK_AMT',
 'IS_SUCCESSFUL']

### What variable(s) are the target(s) for your model?


*   IS_SUCCESSFUL is the target for the model



In [138]:
application_df["IS_SUCCESSFUL"].describe()

count    34299.000000
mean         0.532406
std          0.498956
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: IS_SUCCESSFUL, dtype: float64

### What variable(s) are the feature(s) for your model?

#### There are 7 features for the model,
#### Features:
*   'APPLICATION_TYPE'
* 'AFFILIATION'
* 'CLASSIFICATION'
* 'USE_CASE'
* 'ORGANIZATION'
*  'INCOME_AMT'
*  'ASK_AMT'


In [139]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME", "STATUS","SPECIAL_CONSIDERATIONS"], axis='columns')
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [140]:
# Determine the number of unique values in each column.
application_df.nunique()


APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [141]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df["APPLICATION_TYPE"].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [142]:
# Now change cutoff value to get 11 bins for APPLICATION_TYPE
# Choose a cutoff value and create a list of application types to be replaced
# I picked a cutoff value of 100 to bin: 
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts<50].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()
#  11 Bins

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: APPLICATION_TYPE, dtype: int64

In [120]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df["CLASSIFICATION"].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [143]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts_pos = class_counts[class_counts>1]
class_counts_pos

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [144]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list (class_counts[class_counts<20].index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()
#  22 Bins

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
Other      207
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
Name: CLASSIFICATION, dtype: int64

In [145]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [147]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], axis='columns').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [148]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

### Attempt #1

*   Take out two columns/features: STATUS and SPECIAL_CONSIDERATION
*   Change cutoff values to get 11 bins for APPLICATION_TYPE and 22 bins for CLASSIFICATION.
*   Add two layers from original attempt- Total layers: 5 total
*   Change activation function to for each layer to: relu, relu, tanh, tanh,  sigmoid
*   Number of neurons for each hidden layers: 9,7,7,5,1
*   Epochs: 100



In [149]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  9
hidden_nodes_layer2_3 = 7
hidden_nodes_layer4 = 5

nn_1 = tf.keras.models.Sequential()

# First hidden layer
nn_1.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="relu"))

# Third hidden layer
nn_1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="tanh"))

# Fourth hidden layer
nn_1.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Output layer
nn_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_1.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_67 (Dense)            (None, 9)                 531       
                                                                 
 dense_68 (Dense)            (None, 7)                 70        
                                                                 
 dense_69 (Dense)            (None, 7)                 56        
                                                                 
 dense_70 (Dense)            (None, 5)                 40        
                                                                 
 dense_71 (Dense)            (None, 1)                 6         
                                                                 
Total params: 703
Trainable params: 703
Non-trainable params: 0
_________________________________________________________________


In [150]:
# Compile the model
nn_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [151]:
# Train the model
fit_model = nn_1.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 2ms/step - loss: 0.5962 - accuracy: 0.6987
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5596 - accuracy: 0.7300
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5556 - accuracy: 0.7322
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5542 - accuracy: 0.7317
Epoch 5/100
804/804 [==============================] - 4s 6ms/step - loss: 0.5524 - accuracy: 0.7315
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5514 - accuracy: 0.7332
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5501 - accuracy: 0.7335
Epoch 8/100
804/804 [==============================] - 5s 7ms/step - loss: 0.5498 - accuracy: 0.7334
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5489 - accuracy: 0.7338
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accura

In [152]:
# Evaluate the model using the test data
model_loss_1, model_accuracy_1 = nn_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_1}, Accuracy: {model_accuracy_1}")

268/268 - 1s - loss: 0.5511 - accuracy: 0.7272 - 945ms/epoch - 4ms/step
Loss: 0.5511172413825989, Accuracy: 0.7272303104400635


### Attempt #2


*   Add two layers- 5 total
*   Change activation function to: relu, relu, relu, tanh, sigmoid
*   Number of neurons for each hidden layers: 9, 7,7,5,1



In [154]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  9
hidden_nodes_layer2_3 = 7
hidden_nodes_layer4 = 5

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="relu"))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="relu"))

# Fourth hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_72 (Dense)            (None, 9)                 531       
                                                                 
 dense_73 (Dense)            (None, 7)                 70        
                                                                 
 dense_74 (Dense)            (None, 7)                 56        
                                                                 
 dense_75 (Dense)            (None, 5)                 40        
                                                                 
 dense_76 (Dense)            (None, 1)                 6         
                                                                 
Total params: 703
Trainable params: 703
Non-trainable params: 0
_________________________________________________________________


In [155]:
# Compile the model
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [156]:
# Train the model
fit_model_2 = nn_2.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.6004 - accuracy: 0.6913
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5624 - accuracy: 0.7278
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5569 - accuracy: 0.7304
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5540 - accuracy: 0.7309
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5514 - accuracy: 0.7313
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5507 - accuracy: 0.7327
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7331
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5491 - accuracy: 0.7341
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7338
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5484 - accura

In [157]:
# Evaluate the model using the test data
model_loss_2, model_accuracy_2 = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")

268/268 - 1s - loss: 0.5515 - accuracy: 0.7279 - 653ms/epoch - 2ms/step
Loss: 0.5515285730361938, Accuracy: 0.7279300093650818


### Attempt #3

*    Drop one layer: 5 layers total
*   Activation functions for 4 layers: relu, tanh, tanh, tanh, sigmoid
*   Number of neurons for each hidden layer: 9, 7, 7, 5, 1
*   Increase epochs: 120


In [173]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  9
hidden_nodes_layer2_3 = 7
hidden_nodes_layer4 = 5
hidden_nodes_layer5 = 1

nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="relu"))

# Third hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="relu"))

# # Fourth hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="sigmoid"))

# Check the structure of the model
nn_3.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_101 (Dense)           (None, 9)                 531       
                                                                 
 dense_102 (Dense)           (None, 7)                 70        
                                                                 
 dense_103 (Dense)           (None, 7)                 56        
                                                                 
 dense_104 (Dense)           (None, 5)                 40        
                                                                 
 dense_105 (Dense)           (None, 1)                 6         
                                                                 
Total params: 703
Trainable params: 703
Non-trainable params: 0
_________________________________________________________________


In [174]:
# Compile the model
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [175]:
# Train the model
fit_model_3 = nn_3.fit(X_train_scaled,y_train,epochs=120)

Epoch 1/120
804/804 [==============================] - 5s 3ms/step - loss: 0.6252 - accuracy: 0.6660
Epoch 2/120
804/804 [==============================] - 3s 4ms/step - loss: 0.5673 - accuracy: 0.7271
Epoch 3/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5580 - accuracy: 0.7285
Epoch 4/120
804/804 [==============================] - 2s 3ms/step - loss: 0.5552 - accuracy: 0.7293
Epoch 5/120
804/804 [==============================] - 3s 3ms/step - loss: 0.5524 - accuracy: 0.7319
Epoch 6/120
804/804 [==============================] - 3s 3ms/step - loss: 0.5500 - accuracy: 0.7333
Epoch 7/120
804/804 [==============================] - 3s 3ms/step - loss: 0.5490 - accuracy: 0.7336
Epoch 8/120
804/804 [==============================] - 4s 5ms/step - loss: 0.5483 - accuracy: 0.7351
Epoch 9/120
804/804 [==============================] - 3s 3ms/step - loss: 0.5471 - accuracy: 0.7346
Epoch 10/120
804/804 [==============================] - 3s 3ms/step - loss: 0.5466 - accura

In [176]:
# Evaluate the model using the test data
model_loss_3, model_accuracy_3 = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_3}, Accuracy: {model_accuracy_3}")

268/268 - 1s - loss: 0.5540 - accuracy: 0.7259 - 792ms/epoch - 3ms/step
Loss: 0.5540497899055481, Accuracy: 0.7259474992752075


### Model from attempt #2 was the best but it did not reach 75% accuracy, it only got to 72.79% which was not much better than the original model.

In [177]:
# Evaluate the model using the test data
model_loss_2, model_accuracy_2 = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")

268/268 - 1s - loss: 0.5515 - accuracy: 0.7279 - 888ms/epoch - 3ms/step
Loss: 0.5515285730361938, Accuracy: 0.7279300093650818


### Save the model:

In [178]:
# Export our model to HDF5 file
from google.colab import files

output_filepath = "/content/gdrive/MyDrive/DataClassNotebooks/module_21_deep_learning_challenge/output"

# Save model 2, nn_2, and download a copy to local machine:
nn_2.save('/content/gdrive/MyDrive/DataClassNotebooks/module_21_deep_learning_challenge/output/AlphabetSoupCharity_Optimization.h5')
files.download('/content/gdrive/MyDrive/DataClassNotebooks/module_21_deep_learning_challenge/output/AlphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>